In [1]:
%matplotlib inline
import itertools
import os
os.environ['CUDA_VISIBLE_DEVICES']=""
import numpy as np
import gpflow
import gpflow.training.monitor as mon
import numbers
import matplotlib.pyplot as plt
import tensorflow as tf

# Demo: `gpflow.training.monitor`
In this notebook we'll demo how to use `gpflow.training.monitor` for logging the optimisation of a GPflow model.

## Creating the GPflow model
We first generate some random data and create a GPflow model.

Under the hood, GPflow gives a unique name to each model which is used to name the Variables it creates in the TensorFlow graph containing a random identifier. This is useful in interactive sessions, where people may create a few models, to prevent variables with the same name conflicting. However, when loading the model, we need to make sure that the names of all the variables are exactly the same as in the checkpoint. This is why we pass name="SVGP" to the model constructor, and why we use gpflow.defer_build().

In [2]:
np.random.seed(0)
X = np.random.rand(10000, 1) * 10
Y = np.sin(X) + np.random.randn(*X.shape)
Xt = np.random.rand(10000, 1) * 10
Yt = np.sin(Xt) + np.random.randn(*Xt.shape)

with gpflow.defer_build():
    m = gpflow.models.SVGP(X, Y, gpflow.kernels.RBF(1), gpflow.likelihoods.Gaussian(),
                           Z=np.linspace(0, 10, 5)[:, None],
                           minibatch_size=100, name="SVGP")
    m.likelihood.variance = 0.01
m.compile()

Let's compute log likelihood before the optimisation

In [3]:
print('LML before the optimisation: %f' % m.compute_log_likelihood())

LML before the optimisation: -1271605.621944


We will be using a TensorFlow optimiser. All TensorFlow optimisers have a support for `global_step` variable. Its purpose is to track how many optimisation steps have occurred. It is useful to keep this in a TensorFlow variable as this allows it to be restored together with all the parameters of the model.

The code below creates this variable using a monitor's helper function. It is important to create it before building the monitor in case the monitor includes a checkpoint task. This is because the checkpoint internally uses the TensorFlow Saver which creates a list of variables to save. Therefore all variables expected to be saved by the checkpoint task should exist by the time the task is created.

In [4]:
session = m.enquire_session()
global_step = mon.create_global_step(session)

## Construct the monitor

Next we need to construct the monitor. `gpflow.training.monitor` provides classes that are building blocks for the monitor. Essengially, a monitor is a function that is provided as a callback to an optimiser. It consists of a number of tasks that may be executed at each step, subject to their running condition.

In this example, we want to:
- log certain scalar parameters in TensorBoard,
- log the full optimisation objective (log marginal likelihood bound) periodically, even though we optimise with minibatches,
- store a backup of the optimisation process periodically,
- log performance for a test set periodically.

We will define these tasks as follows:

In [5]:
print_task = mon.PrintTimingsTask().with_name('print')\
    .with_condition(mon.PeriodicIterationCondition(10))\
    .with_exit_condition(True)

sleep_task = mon.SleepTask(0.01).with_name('sleep').with_name('sleep')

saver_task = mon.CheckpointTask('./monitor-saves').with_name('saver')\
    .with_condition(mon.PeriodicIterationCondition(10))\
    .with_exit_condition(True)

std_tboard_task = mon.StandardTensorBoardTask(m, './model-tensorboard').with_name('std_tboard')\
    .with_condition(mon.PeriodicIterationCondition(10))\
    .with_exit_condition(True)

lml_tboard_task = mon.LmlTensorBoardTask(m, './model-tensorboard').with_name('lml_tboard')\
    .with_condition(mon.PeriodicIterationCondition(100))\
    .with_exit_condition(True)

As the above code shows, each task can be assigned a name and running conditions. The name will be shown in the task timing summary.

There are two different types of running conditions: `with_condition` controls execution of the task at each iteration in the optimisation loop. `with_exit_condition` is a simple boolean flag indicating that the task should also run at the end of optimisation.
In this example we want to run our tasks periodically, at every iteration or every 10th or 100th iteration.


## Custom tasks
We may also want to perfom certain tasks that do not have pre-defined `Task` classes. For example, we may want to compute the performance on a test set. Here we create such a class by extending `BaseTensorBoardTask` to log the testing benchmarks in addition to all the scalar parameters.

In [6]:
class CustomTensorBoardTask(mon.BaseTensorBoardTask):
    def __init__(self, model, event_path, Xt, Yt):
        super().__init__(model, event_path)
        self.Xt = Xt
        self.Yt = Yt
        self._full_test_err = tf.placeholder(gpflow.settings.tf_float, shape=())
        self._full_test_nlpp = tf.placeholder(gpflow.settings.tf_float, shape=())
        self._summary = tf.summary.merge([tf.summary.scalar("test_rmse", self._full_test_err),
                                         tf.summary.scalar("test_nlpp", self._full_test_nlpp)])
    
    def run(self, context: mon.MonitorContext, *args, **kwargs) -> None:
        minibatch_size = 100
        preds = np.vstack([self.model.predict_y(Xt[mb * minibatch_size:(mb + 1) * minibatch_size, :])[0]
                            for mb in range(-(-len(Xt) // minibatch_size))])
        test_err = np.mean((Yt - preds) ** 2.0)**0.5
        self._eval_summary(context, {self._full_test_err: test_err, self._full_test_nlpp: 0.0})

        
custom_tboard_task = CustomTensorBoardTask(m, './model-tensorboard', Xt, Yt).with_name('custom_tboard')\
    .with_condition(mon.PeriodicIterationCondition(100))\
    .with_exit_condition(True)

Now we can put all these tasks into a monitor.

In [7]:
monitor_tasks = [print_task, std_tboard_task, lml_tboard_task, custom_tboard_task, saver_task, sleep_task]
monitor = mon.Monitor(monitor_tasks, session, global_step)

## Running the optimisation
We finally get to running the optimisation.

We may want to continue a previously run optimisation by resotring the TensorFlow graph from the latest checkpoint. Otherwise skip this step.

In [8]:
mon.restore_session(session, './monitor-saves')

Restoring session from `./monitor-saves/cp-450`.
INFO:tensorflow:Restoring parameters from ./monitor-saves/cp-450


Now we will run the optimisation with attached monitor. At the end we want to print the summary of time spent on each task.

In [9]:
try:
    optimiser = gpflow.train.AdamOptimizer(0.01)
    monitor.start_monitoring()
    optimiser.minimize(m, step_callback=monitor, maxiter=450, global_step=global_step)
finally:
    monitor.stop_monitoring()
    monitor.print_summary()

Iteration 10	total itr.rate 13.04/s	recent itr.rate 13.04/s	opt.step 460	total opt.rate 14.79/s	recent opt.rate 14.79/s
Iteration 20	total itr.rate 20.03/s	recent itr.rate 43.23/s	opt.step 470	total opt.rate 28.96/s	recent opt.rate 696.70/s
Iteration 30	total itr.rate 25.75/s	recent itr.rate 60.05/s	opt.step 480	total opt.rate 42.61/s	recent opt.rate 738.60/s
Iteration 40	total itr.rate 29.03/s	recent itr.rate 46.91/s	opt.step 490	total opt.rate 55.73/s	recent opt.rate 734.90/s
Iteration 50	total itr.rate 31.45/s	recent itr.rate 47.19/s	opt.step 500	total opt.rate 68.46/s	recent opt.rate 792.80/s
Iteration 60	total itr.rate 33.48/s	recent itr.rate 49.46/s	opt.step 510	total opt.rate 80.17/s	recent opt.rate 551.65/s
Iteration 70	total itr.rate 35.01/s	recent itr.rate 48.23/s	opt.step 520	total opt.rate 92.05/s	recent opt.rate 833.51/s
Iteration 80	total itr.rate 35.35/s	recent itr.rate 37.97/s	opt.step 530	total opt.rate 103.46/s	recent opt.rate 780.75/s
Iteration 90	total itr.rate 36.3

100%|██████████| 100/100 [00:00<00:00, 390.33it/s]


Iteration 110	total itr.rate 31.80/s	recent itr.rate 12.71/s	opt.step 560	total opt.rate 132.29/s	recent opt.rate 481.63/s
Iteration 120	total itr.rate 32.71/s	recent itr.rate 47.56/s	opt.step 570	total opt.rate 142.31/s	recent opt.rate 856.74/s
Iteration 130	total itr.rate 33.43/s	recent itr.rate 45.45/s	opt.step 580	total opt.rate 150.56/s	recent opt.rate 494.70/s
Iteration 140	total itr.rate 34.07/s	recent itr.rate 45.29/s	opt.step 590	total opt.rate 159.17/s	recent opt.rate 620.22/s
Iteration 150	total itr.rate 34.69/s	recent itr.rate 46.60/s	opt.step 600	total opt.rate 166.65/s	recent opt.rate 486.82/s
Iteration 160	total itr.rate 35.31/s	recent itr.rate 48.30/s	opt.step 610	total opt.rate 175.38/s	recent opt.rate 818.22/s
Iteration 170	total itr.rate 35.84/s	recent itr.rate 47.18/s	opt.step 620	total opt.rate 183.40/s	recent opt.rate 684.53/s
Iteration 180	total itr.rate 36.30/s	recent itr.rate 46.52/s	opt.step 630	total opt.rate 190.10/s	recent opt.rate 501.59/s
Iteration 190	to

 14%|█▍        | 14/100 [00:00<00:00, 132.16it/s]

Iteration 200	total itr.rate 37.13/s	recent itr.rate 45.21/s	opt.step 650	total opt.rate 205.71/s	recent opt.rate 753.61/s


...

Iteration 410	total itr.rate 36.11/s	recent itr.rate 15.74/s	opt.step 860	total opt.rate 310.23/s	recent opt.rate 710.03/s
Iteration 420	total itr.rate 36.31/s	recent itr.rate 47.26/s	opt.step 870	total opt.rate 314.44/s	recent opt.rate 709.10/s
Iteration 430	total itr.rate 36.48/s	recent itr.rate 45.72/s	opt.step 880	total opt.rate 317.20/s	recent opt.rate 502.02/s
Iteration 440	total itr.rate 36.68/s	recent itr.rate 47.82/s	opt.step 890	total opt.rate 319.88/s	recent opt.rate 502.22/s
Iteration 450	total itr.rate 36.84/s	recent itr.rate 45.79/s	opt.step 900	total opt.rate 321.88/s	recent opt.rate 444.05/s


 49%|████▉     | 49/100 [00:00<00:00, 488.51it/s]

Iteration 450	total itr.rate 36.09/s	recent itr.rate 0.00/s	opt.step 900	total opt.rate 290.59/s	recent opt.rate 0.00/s


100%|██████████| 100/100 [00:00<00:00, 482.34it/s]


Tasks execution time summary:
print:	0.0433 (sec)
std_tboard:	0.1348 (sec)
lml_tboard:	1.5571 (sec)
custom_tboard:	1.1556 (sec)
saver:	3.9401 (sec)
sleep:	4.5420 (sec)


Now lets compute the log likelihood again. Hopefully we will see an increase in its value

In [10]:
print('LML after the optimisation: %f' % m.compute_log_likelihood())

LML after the optimisation: -15550.138259
